# Use my_net multi input version

# 1. Data Clean

## Define data clean function
if you need to do zero center, you should use data_normal rather than data_clean.

In [1]:

def data_normal(X_,y_,is_training):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in X_.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1+band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
        
    X_data_angle=np.reshape(X_["inc_angle"],(X_["inc_angle"].shape[0],1))
    if is_training:
        y_data = keras.utils.to_categorical(y_, num_classes)
        
    else:
        y_data = None
    return (np.array(images),X_data_angle,y_data)

def data_clean(X_,y_,is_training):
    
    X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in X_["band_1"]])
    X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in X_["band_2"]])

    X_data = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                              X_band_2[:, :, :, np.newaxis], 
                              ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], 
                             axis=-1)
    X_data_angle=np.reshape(X_["inc_angle"],(X_["inc_angle"].shape[0],1))
    if is_training:
        y_data = keras.utils.to_categorical(y_, num_classes)
    else:
        y_data = None
    return (X_data,X_data_angle,y_data)


## Define threadsafe_generator 
If we want to use data augmentation in training process,we have to use @threadsafe_generator to make any generator thread safe.

Note: A non thread safe generator in a multithreaded envrionment just crashes, yieldsing a  'ValueError: generator already executing' error.You can try it yourself by removing the @threadsafe_generator decorator from count().

Also, since we have to use multiple generators to feed both the image data and the angle data, we have to define our own generator: generator_img_angle()

## Do Data Cleaning on training data and validation data

In [3]:
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import keras
import math
#Load the data.
train = pd.read_json("../input/train.json")

#use mean of angle to replace "na"
inc_angle = train.inc_angle.replace('na',0)
idx=np.where(inc_angle==0)
inc_angle = inc_angle.drop(idx[0])
inc_mean=np.mean(inc_angle)
train.inc_angle = train.inc_angle.replace('na',inc_mean)
# train.inc_angle = train.inc_angle.replace('na',0)
num_classes=2


y_train=train['is_iceberg']

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(train,y_train, random_state=1, train_size=0.75)
X_train_cv,X_train_angle,y_train_cv=data_clean(X_train_cv,y_train_cv,is_training=True)
X_valid,X_valid_angle,y_valid=data_clean(X_valid,y_valid,is_training=True)

# X_train_cv,X_train_angle,y_train_cv=data_normal(X_train_cv,y_train_cv,is_training=True)
# X_valid,X_valid_angle,y_valid=data_normal(X_valid,y_valid,is_training=True)

print(X_train_cv.shape,X_train_angle.shape,y_train_cv.shape)
print(X_valid.shape,X_valid_angle.shape,y_valid.shape)

/home/cv/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(1203, 75, 75, 3) (1203, 1) (1203, 2)
(401, 75, 75, 3) (401, 1) (401, 2)


/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


## Just for fun, post a tweet when the val_acc has been updated

If you want to use this function, you have to add my_call_back in "callbacks" list

In [2]:
import threading
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
 
    def __iter__(self):
        return self
 
    def __next__(self):
        with self.lock:
            return self.it.__next__()
 
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g
 

def plt_images(X_images):
    fig = plt.figure(200, figsize=(15, 15))
    random_indicies = np.random.choice(range(len(X_images)), 9, False)
    subset = X_images[random_indicies]
    for i in range(9):
        ax = fig.add_subplot(3, 3, i + 1)
        ax.imshow(subset[i])
    plt.show()
    
@threadsafe_generator
def generator_img_angle(X1, X2, Y,batch_size=32,is_training=True):
#     index=1
    while True:
        if is_training:
            idx = np.random.permutation( X1.shape[0])
        else:
            idx =range(X1.shape[0])
        datagen = ImageDataGenerator(featurewise_center=True,  # set input mean to 0 over the dataset
                                     samplewise_center=False,  # set each sample mean to 0
                                     featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                     samplewise_std_normalization=False,  # divide each input by its std
                                     zca_whitening=False,  # apply ZCA whitening
                                     rotation_range=False,  # randomly rotate images in the range (degrees, 0 to 180)
                                     width_shift_range=False,  # randomly shift images horizontally (fraction of total width)
                                     height_shift_range=False,  # randomly shift images vertically (fraction of total height)
                                     horizontal_flip=False,  # randomly flip images
                                     vertical_flip=False)  # randomly flip image
        datagen.fit(X1)
        if is_training:
            batches = datagen.flow(X1,Y,batch_size=batch_size,shuffle=False)
        else:
            batches = datagen.flow(X1,batch_size=batch_size,shuffle=False)
        idx0 = 0
#         print("batch num:",len(batches))
        for batch in batches:
#             print("\nidx:%d\n"%index)
#             print("batch numi:",len(batch))
#             index+=1
            if is_training:
                idx1 = idx0 + batch[0].shape[0]
            else:
                idx1=idx0 + batch.shape[0]
            # plot images
#             X_images=batch[0]
#             plt_images(X_images)
            if is_training:
                yield [batch[0], X2[ idx[ idx0:idx1 ] ]], batch[1]
            else:
                yield [batch, X2[ idx[ idx0:idx1 ] ]]
            idx0 = idx1
            if idx1 >= X1.shape[0]:
                break


Using TensorFlow backend.


In [11]:
from urllib import parse,request
headers = {
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
"Accept-Encoding":"gzip, deflate, br",
"Accept-Language":"zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7",
"Cache-Control":"max-age=0",
"Connection":"keep-alive",
"Content-Type":"application/x-www-form-urlencoded",
"Cookie":"_T_WM=977f85fa803c66b53d32bc3154488755; ALF=1517195081; SCF=Ar3eydUgzdi_dflitqApnqkIrqkmy2KVgbt_DLDlyqKCaF-qLn8iFGjp1fFELeOstkWfTkyH-OPVA3trocoCkLA.; SUB=_2A253Q1QnDeRhGeBO71oR9i3Fwj-IHXVUzHxvrDV6PUJbktANLVDfkW1NRew5lIyze7lEjwMqZII2Q3uZToA6X0BI; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5lYHIxViFXVlcOoSRecgrN5JpX5K-hUgL.Foq7Shn7Soe41Ke2dJLoI7v.qg4oIg4rKsLu9-pDdJjt; SUHB=0P1MbOkE2VISXc; SSOLoginState=1514611831",
"Host":"weibo.cn",
"Origin":"https://weibo.cn",
"Referer":"https://weibo.cn/",
"Upgrade-Insecure-Requests":"1",
"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/63.0.3239.84 Chrome/63.0.3239.84 Safari/537.36"
}

url="https://weibo.cn/mblog/sendmblog?st=431d54"
# if val_acc update, it will update a weibo 
class My_call_back(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.best_val_acc=0.87

    def on_epoch_end(self, epoch,logs={}):
        now_val_acc=logs.get('val_acc')
        if now_val_acc>self.best_val_acc:
            self.best_val_acc=now_val_acc
            #post a tweet
            content="epoch %d has ended,best_val_acc has updated, now it is : %.10lf\n"%(epoch+1,self.best_val_acc)
            print()
            print(content)
            post_data ={
                "rl":"0",
                "content":content
            }
            post_data = parse.urlencode(post_data).encode('utf-8')
            req = request.Request(url=url,data=post_data,headers=headers)
            res = request.urlopen(req).read()

my_call_back = My_call_back()   

# 2. Train model

If we use data augmentaion, use the model.fit_generator version, otherwise use model.fit

In [12]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalMaxPooling2D
import os
from keras import regularizers
import math
from keras.layers import advanced_activations

#Macro defined
batch_size =64
num_classes =2
epochs = 80
batch_num_per_epoch=math.ceil(X_train_cv.shape[0]/batch_size)
batch_num_val=math.ceil(X_valid.shape[0]/batch_size)
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_iceberg_myownmodel2.h5'
filepath="keras_iceberg_epoch_mymodel.h5"

# create the base model
def conv2d_block(inputs,kernel_size,filters):
    x=Conv2D(filters=filters,kernel_size= kernel_size,strides=(1, 1), padding='same',
           input_shape=X_train_cv.shape[1:],kernel_initializer='glorot_normal',
           use_bias=True,kernel_regularizer=regularizers.l2(0.03))(inputs)
    
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, 
                                scale=True, beta_initializer='zeros', gamma_initializer='ones', 
                                moving_mean_initializer='zeros', moving_variance_initializer='ones')(x)
#     x=advanced_activations.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)(x)
    x=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.2)(x)

    return x

def dense_block(x,filters):
    x=Dense(filters,kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.03))(x)
    x=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
    x=Dropout(0.2)(x)
    
    return x
# This returns a tensor
main_input = Input(shape=X_train_cv.shape[1:],name='image_input')
x=main_input
# x=keras.layers.convolutional.Cropping2D(cropping=((10, 10), (10, 10)), data_format=None)(x)

print(x.shape)

# a layer instance is callable on a tensor, and returns a tensor
# x = conv2d33_maxpool_block(main_input,(3,3),64)
x = conv2d_block(x,(3,3),128)
x = conv2d_block(x,(3,3),256)
x = conv2d_block(x,(3,3),512)

# You must flatten the data for the dense layers
# x=Flatten()(x)
# x=Dense(512)(x)
# x = conv2d33_maxpool_block(x,(1,1),256)

image_out=GlobalMaxPooling2D()(x)
x=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
# x=advanced_activations.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)(x)
x=Dropout(0.2)(x)

#add time
angle_input = Input(shape=(1,), name='angle_input')
x = keras.layers.concatenate([image_out, angle_input],axis=-1)
x=dense_block(x,256)
x=dense_block(x,64)
x=dense_block(x,16)

# x=advanced_activations.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)(x)
predictions = Dense(num_classes, activation='sigmoid',name='main_output',kernel_initializer='glorot_normal',
                   kernel_regularizer=regularizers.l2(0.03))(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[main_input, angle_input], outputs=predictions)

opt=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#input data

# Fit the model on the batches generated by datagen.flow().
model.fit({'image_input': X_train_cv, 'angle_input':  X_train_angle},
          {'main_output': y_train_cv},
          validation_data=([X_valid,X_valid_angle],y_valid),
          epochs=epochs, batch_size=batch_size,
          callbacks=[keras.callbacks.ModelCheckpoint(filepath,
                                                        monitor='val_acc',
                                                        verbose=0,
                                                        save_best_only=True, 
                                                        mode='auto')])

# model.fit_generator(generator_img_angle(X_train_cv, X_train_angle, y_train_cv,batch_size,is_training=True),
#                                     epochs=epochs,
# #                                     validation_data=([X_valid,X_valid_angle],y_valid),
#                                     validation_data=generator_img_angle(X_valid,X_valid_angle,y_valid,
#                                                                         batch_size,is_training=True),
#                                     validation_steps=batch_num_val,
#                                     workers=4,
#                                     steps_per_epoch=batch_num_per_epoch,
#                                     # 该回调函数将在每个epoch后保存模型到filepath
#                                     callbacks=[keras.callbacks.ModelCheckpoint(filepath,
#                                                         monitor='val_acc',
#                                                         verbose=0,
#                                                         save_best_only=True, 
#                                                         mode='auto')])
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate([X_valid, X_valid_angle],y_valid, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

(?, 75, 75, 3)
Train on 1203 samples, validate on 401 samples
Epoch 1/80
1203/1203 [==============================] - 6s 5ms/step - loss: 20.1225 - acc: 0.5428 - val_loss: 15.0000 - val_acc: 0.5312
Epoch 2/80
1203/1203 [==============================] - 4s 3ms/step - loss: 11.5392 - acc: 0.5278 - val_loss: 8.4563 - val_acc: 0.5312
Epoch 3/80
1203/1203 [==============================] - 4s 3ms/step - loss: 6.7668 - acc: 0.5387 - val_loss: 5.3498 - val_acc: 0.5312
Epoch 4/80
1203/1203 [==============================] - 4s 3ms/step - loss: 4.4067 - acc: 0.5993 - val_loss: 3.7574 - val_acc: 0.6983
Epoch 5/80
1203/1203 [==============================] - 4s 3ms/step - loss: 3.1330 - acc: 0.7531 - val_loss: 2.8815 - val_acc: 0.5810
Epoch 6/80
1203/1203 [==============================] - 4s 3ms/step - loss: 2.4074 - acc: 0.7805 - val_loss: 2.3218 - val_acc: 0.7431
Epoch 7/80
1203/1203 [==============================] - 4s 3ms/step - loss: 1.9655 - acc: 0.7972 - val_loss: 1.9925 - val_acc: 0.55

## validation check

In [15]:
import keras
# Score trained model.
scores = model.evaluate([X_valid, X_valid_angle],y_valid, verbose=1)
# scores = model.evaluate_generator(generator_img_angle(X_valid, X_valid_angle, y_valid,batch_size=batch_size),
#                                   steps=batch_num_val)

print("validation accuracy:",scores)

401/401 [==============================] - 1s 2ms/step
validation accuracy: [0.56922518835400704, 0.88279301819955913]


# 3.Predict

## Load a pretrained model

In [13]:
from keras.models import load_model
model_path="../iceberg_code/keras_iceberg_epoch_mymodel.h5"
model = load_model(model_path)  

## predict

In [25]:
import json
import numpy as np
import pandas as pd
import math

cut_set_num=10
lenth_test=8424
num_classes=2
is_training=False
test_path="../input/test_cut/"
batch_size=32


predicted_test=[]
result_max=[]
test_id=[]


for ci in range(cut_set_num):
    test_pathi=test_path+"test%d.json"%ci
    test_json_str = open(test_pathi, 'r').readlines()[0]
    test=pd.read_json(test_json_str)
    
    #use mean of angle to replace "na"
    inc_angle = test.inc_angle.replace('na',0)
    idx=np.where(inc_angle==0)
    inc_angle = inc_angle.drop(idx[0])
    inc_mean=np.mean(inc_angle)
    test.inc_angle = test.inc_angle.replace('na',inc_mean)
#     test.inc_angle = test.inc_angle.replace('na',0)
   
    X_test,X_test_angle,y_test=data_normal(X_=test,y_=None,is_training=False)
    batch_num_predic=math.ceil(X_test.shape[0]/batch_size)
#     print(X_test.shape[0],batch_num_predic)
    #发现一个问题,steps总是比我设置的多出10
    predicted_testi=model.predict(X_test)
    
#     predicted_testi=model.predict_generator(generator_img_angle(X1=X_test,
#                                                                 X2=X_test_angle, 
#                                                                 Y=None,
#                                                                 batch_size=batch_size,
#                                                                 is_training=False),
#                                             steps=batch_num_predic)
    
    test_id.extend(test["id"])
    predicted_testi=predicted_testi.reshape(predicted_testi.shape[0],2)
    for index in range(len(predicted_testi)):
        predicted_test.append(predicted_testi[index][1])
    result_max.extend( np.argmax(predicted_testi, axis = 1) )
    
    print("batch%d has finished!"%ci)
    print()

/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


batch0 has finished!

batch1 has finished!

batch2 has finished!

batch3 has finished!

batch4 has finished!

batch5 has finished!

batch6 has finished!

batch7 has finished!

batch8 has finished!

batch9 has finished!



In [26]:
# save result
submission = pd.DataFrame()
print(len(test_id),len(result_max),len(predicted_test))
submission['id']=test_id
submission['is_iceberg']=result_max
submission.to_csv('sub.csv', index=False)
# save probablity
submission['is_iceberg']=predicted_test
submission.to_csv('simple_model.csv', index=False,float_format="%.8lf")


8424 8424 8424


In [11]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 75, 75, 128)  3584        image_input[0][0]                
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 75, 75, 128)  512         conv2d_13[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_28 (LeakyReLU)      (None, 75, 75, 128)  0           batch_normalization_13[0][0]     
__________________________________________________________________________________________________
max_poolin

In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.